# Notebook for Loading in the Database and Creating a Graph

Start by importing the libraries and connecting to the neo4j sandbox.

In [1]:
# imports
import pandas as pd
from graphdatascience import GraphDataScience

/home/seb_s_ben/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Connect to neo4j DBMS

#this is local DBMS via neo4j Desktop for testing purposes needs to be changed
DB_ULR = "bolt://localhost:11004" 
DB_USER = "neo4j"
DB_PASS = "1234"
DB_NAME = "neo4j"
gds = GraphDataScience(DB_ULR, auth=(DB_USER, DB_PASS))

## Loading data into the graph

Preprocessed data stored in https://github.com/EY-Tech-Consulting-Denmark/Graphathon-ATP/tree/main/Data/clean_data can be loaded into a Neo4j sandbox.

### Loading the preprocessed data

In [3]:
data = pd.read_csv("../Data/clean_data/data.csv") # pd.read_csv("https://raw.githubusercontent.com/EY-Tech-Consulting-Denmark/Graphathon-ATP/main/Data/clean_data/data.csv")
data.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ClmProcedureCode_5,PotentialFraud,Age,IsDead,DaysAdmitted,DaysClaimLasted,ClaimEndAfterDischarged,TotalDiagnosis,TotalProcedures,TotalPhysicians
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,NaN,1,66.0,0,7,7,0,9,0,1
1,BENE17521,1913-12-01,NaN,2,1,0,39,230,12,12,...,NaN,1,95.0,0,13,13,0,9,2,3
2,BENE21718,1922-10-01,NaN,1,1,0,39,600,12,12,...,NaN,1,87.0,0,19,19,0,9,1,2
3,BENE22934,1930-07-01,NaN,2,1,0,39,280,12,12,...,NaN,1,79.0,0,5,5,0,7,1,3
4,BENE24402,1925-09-01,NaN,2,2,0,39,510,12,12,...,NaN,1,83.0,0,5,5,0,1,1,2


## Create the nodes in the graph
Start with creating beneficiary nodes and attaching the chronic conditions they have as relationships (includes renal disease)

In [4]:
# Create the Condition nodes 
gds.run_cypher('create constraint if not exists for (n:Condition) require (n.id) is node key')
gds.run_cypher('''
    unwind [
        'RenalDiseaseIndicator',
        'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
        'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
        'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
        'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
        'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
        'ChronicCond_stroke'
    ] as conditionId
    merge (n:Condition{id: conditionId})
''')

""


In [5]:
# Create Beneficiary nodes and connect them with HAS_CONDITION relationships to their corrosponding chronic conditions
gds.run_cypher('create constraint if not exists for (n:Beneficiary) require (n.id) is node key')
gds.run_cypher('''
    match (c:Condition)
    with collect(c) as conditions
    unwind $data as row
    merge (n:Beneficiary{id: row.BeneID})
        set n.dob = date(row.DOB),
            n.gender = row.Gender,
            n.race = row.Race,
            n.age = row.Age,
            n.state = row.State,
            n.county = row.County
    with conditions, n, row
    call {
        with row, conditions, n
        foreach(
            c in [x in conditions where row[x.id] = 1 or row[x.id] = 'Y' | x] |
            merge (n)-[:HAS_CONDITION]->(c)
        )
    }
''', params = {'data': data.to_dict('records')})

""


In [6]:
# Add date of death (DOD) to beneficiaries who have passed away

dead = data[['BeneID','DOD']].dropna()
gds.run_cypher(''' 
    unwind $data as row
    match (n:Beneficiary{id: row.BeneID})
        set n.dod = date(row.DOD)
''', params = {'data': dead.to_dict('records')})

""


In [7]:
# Create Provider nodes

gds.run_cypher('create constraint if not exists for (n:Provider) require (n.id) is node key')
label_dist = gds.run_cypher('''
    unwind $data as row
    merge (n:Provider{id: row.Provider})
        set n.fraud = case row.PotentialFraud when 1 then true else false end
    return n.fraud as is_fraud, count(*) as count
''', params = {'data': data.to_dict('records')})
label_dist.head()


,is_fraud,count
0,True,23402
1,False,17072


### Create Claims Data

Going to create a node for each Claim, Provider, Physician, Diagnosis, and Procedure code. Create the relationships between beneficiary and their claims.

In [9]:
# Create claims data

# Create constraints on our new node labels
gds.run_cypher('create constraint if not exists for (n:Claim) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Provider) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Physician) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Diagnosis) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Procedure) require (n.id) is node key')

claims_cypher = '''
    unwind $data as row
    merge (c:Claim{id: row.ClaimID})
        set c.startDate = date(row.ClaimStartDt),
            c.endDate = date(row.ClaimEndDt),
            c.admissionDate = date(row.AdmissionDt),
            c.dischargeDate = date(row.DischargeDt),
            c.deductibleAmt = row.DeductibleAmtPaid,
            c.reimbursedAmt = row.InscClaimAmtReimbursed
            c.daysAdmitted = row.DaysAdmitted
            c.daysClaimLasted = row.DaysClaimLasted
            c.claimEndAfterDischarged = row.ClaimEndAfterDischarged 
    merge (p:Provider{id: row.Provider})
    merge (c)-[:PROVIDER]->(p)
    merge (b:Beneficiary{id: row.BeneID})
    merge (c)-[:BENEFICIARY]->(b)
    '''

gds.run_cypher(claims_cypher, params = {'data': data.to_dict('records')})

""


In [10]:
# Create Diagnoses code nodes and create the relationship ADMIT_DIAGNOSIS

gds.run_cypher('''
    unwind $data as row
    match (c:Claim)
    where c.id = row.ClaimID
    merge (d:Diagnosis{id: row.ClmAdmitDiagnosisCode})
    merge (c)-[:ADMIT_DIAGNOSIS]->(d)
''', params = {'data': data.to_dict('records')})

""


In [11]:
# Create Diagnoses code nodes and create the relationship ADMIT_DIAGNOSIS

gds.run_cypher('''
    unwind $data as row
    match (c:Claim)
    where c.id = row.ClaimID
    merge (d:Diagnosis{id: row.DiagnosisGroupCode})
    merge (c)-[:DIAGNOSIS_GROUP]->(d)
''', params = {'data': data.to_dict('records')})

""


In [26]:
# Create the relationships between claims and claim diagnoses codes
gds.run_cypher('''
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    with c, row, [ x in  [ 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
                            'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
                            'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
                            'ClmDiagnosisCode_10' ] where not row[x] is null |
                   row[x]] as codes
    foreach(
        code in codes | 
        merge (d:Diagnosis{id: code})
        merge (c)-[:DIAGNOSIS_CODE]->(d)
    )
''', params = {'data': data.to_dict('records')})

""


### Add Procedure nodes and connect them with the claim nodes

In [27]:
# Create the relationships between claims and procedure codes

gds.run_cypher('''
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    with c, row, [ x in  [ 'ClmProcedureCode_1', 'ClmProcedureCode_2',
                            'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
                            'ClmProcedureCode_6' ] where not row[x] is null |
                   row[x]] as codes
    foreach(
        code in codes | 
        merge (p:Procedure{id: code})
        merge (c)-[:PROCEDURE_CODE]->(p)
    )
''', params = {'data': data.to_dict('records')})

""


### Add the Physician nodes

In [22]:
# Add Physicians and their realtionship to the claim

# Add attending physicians
attend = data[['ClaimID','AttendingPhysician']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (n:Claim{id: row.ClaimID})
    merge (ap:Physician{id: row.AttendingPhysician})
    merge (n)-[:ATTENDING]->(ap)
''', params = {'data': attend.to_dict('records')})

""


In [23]:
# Add operating physicians
operating = data[['ClaimID','OperatingPhysician']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (n:Claim{id: row.ClaimID})
    merge (ap:Physician{id: row.OperatingPhysician})
    merge (n)-[:OPERATING]->(ap)
''', params = {'data': operating.to_dict('records')})

""


In [24]:
# Add other physicians
other = data[['ClaimID','OtherPhysician']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (n:Claim{id: row.ClaimID})
    merge (ap:Physician{id: row.OtherPhysician})
    merge (n)-[:OTHER]->(ap)
''', params = {'data': other.to_dict('records')})

""


This marks the end of data loading here is the final schema of the data. To see this in the sandbox execute: CALL db.schema.visualization()